## croplearn.ipynb

In [1]:
import sys
sys.path.insert(0,'../cropseg/')

datasetinfo = { "datadir":"/home/ucfaab0/Desktop/su_african_crops_ghana/",
                "metadatadir":"/home/ucfaab0/Desktop/su_african_crops_ghana/metadata/",
                "dataset":"su_african_crops_ghana",
                "groundcollection":"su_african_crops_ghana_labels",
                "s1collection":"su_african_crops_ghana_source_s1",
                "s2collection":"su_african_crops_ghana_source_s2",
                "groundlabels":"su_african_crops_ghana_labels_id.json",
                "groundmetadata":"su_african_crops_ghana_labels.json",
                "s1metadata":"su_african_crops_ghana_source_s1.json",
                "s2metadata":"su_african_crops_ghana_source_s2.json",
                "groundname":"labels.tif",
                "s1imagename":"source.tif",
                "s2imagename":"source.tif",
                "s2maskname":"cloudmask.tif",
                "groundshape":[64,64],
                "s1shape":[64,64],
                "s2shape":[64,64],
                "extension":"tif"
              }
s1bands = [
            {"band":"vv","idx":0},
            {"band":"vh","idx":1},    
          ]  
s2bands = [
            {"band":"blue","wavelength":490,"idx":0},
            {"band":"green","wavelength":560,"idx":1},
            {"band":"red","wavelength":665,"idx":2},
            {"band":"rded1","wavelength":705,"idx":3},
            {"band":"rded2","wavelength":740,"idx":4},
            {"band":"rded3","wavelength":783,"idx":5},
            {"band":"nir","wavelength":842,"idx":6},
            {"band":"rded4","wavelength":865,"idx":7},
            {"band":"swir1","wavelength":1610,"idx":8},
            {"band":"swir2","wavelength":2190,"idx":9}
          ]
s1indices = ["vhvv"]
s2indices = ["ndvi","rdedci","ndmi"]

from mlhubdata import loadjson
groundlabels = loadjson(f'{datasetinfo["metadatadir"]}{datasetinfo["groundlabels"]}')
groundmetadata = loadjson(f'{datasetinfo["metadatadir"]}{datasetinfo["groundmetadata"]}')
s1metadata = loadjson(f'{datasetinfo["metadatadir"]}{datasetinfo["s1metadata"]}')
s2metadata = loadjson(f'{datasetinfo["metadatadir"]}{datasetinfo["s2metadata"]}')

skiplist = ["001268"]

In [3]:
d = numpy.copy(data)
numpy.random.shuffle(d)
d[:,0] = d[:,0] - 1
d0 = d[d[:,0] == 0][:40000]
d1 = d[d[:,0] == 1][:40000]
d2 = d[d[:,0] == 2][:40000]
d3 = d[d[:,0] == 3][:40000]
d = numpy.concatenate([d0,d1,d2,d3])
numpy.random.shuffle(d)
xtrain, ytrain = d[4000:,1:],d[4000:,0] 
xtest, ytest = d[:4000,1:],d[:4000,0] 

In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
model = tf.keras.Sequential([
    layers.Dense(dctcoefficients*2,activation='relu'),
    layers.Dense(dctcoefficients*2,activation='relu'),
    layers.Dense(4)
])
model.compile(optimizer='Adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(xtrain,ytrain,epochs=21)
model.summary()

Epoch 1/21
4875/4875 [==============================] - 5s 960us/step - loss: 1.6480 - accuracy: 0.3483
Epoch 2/21
4875/4875 [==============================] - 5s 956us/step - loss: 1.2987 - accuracy: 0.3862
Epoch 3/21
4875/4875 [==============================] - 5s 951us/step - loss: 1.2877 - accuracy: 0.3941
Epoch 4/21
4875/4875 [==============================] - 5s 964us/step - loss: 1.2839 - accuracy: 0.3953
Epoch 5/21
4875/4875 [==============================] - 5s 956us/step - loss: 1.2814 - accuracy: 0.3984
Epoch 6/21
4875/4875 [==============================] - 5s 959us/step - loss: 1.2784 - accuracy: 0.3998
Epoch 7/21
4875/4875 [==============================] - 5s 951us/step - loss: 1.2792 - accuracy: 0.3996
Epoch 8/21
4875/4875 [==============================] - 5s 962us/step - loss: 1.2751 - accuracy: 0.4013
Epoch 9/21
4875/4875 [==============================] - 5s 961us/step - loss: 1.2773 - accuracy: 0.4013
Epoch 10/21
4875/4875 [==============================] - 5s 1ms/

In [5]:
import sklearn.metrics
model.evaluate(xtest,ytest,verbose=1)
predictions = model.predict(xtest)
predict = numpy.argmax(predictions,axis = 1)
true = numpy.array(ytest,dtype=int)
print(sklearn.metrics.f1_score(true,predict,average=None),sklearn.metrics.f1_score(true,predict,average="weighted"))
print(sklearn.metrics.cohen_kappa_score(true,predict))

125/125 [==============================] - 0s 612us/step - loss: 1.2712 - accuracy: 0.4040
[0.45812124 0.15564202 0.4745167  0.42230026] 0.3770130412795332
0.20603506600749366


In [6]:
import sklearn.dummy
dummy = sklearn.dummy.DummyClassifier(strategy="uniform")
dummy.fit(xtrain,ytrain)
predict = dummy.predict(xtest)
true = numpy.array(ytest,dtype=int)
print(sklearn.metrics.accuracy_score(true,predict))
print(sklearn.metrics.f1_score(true,predict,average=None),sklearn.metrics.f1_score(true,predict,average="weighted"))
print(sklearn.metrics.cohen_kappa_score(true,predict))

0.25675
[0.244      0.26013007 0.26821862 0.25481481] 0.2567395848828598
0.009034766363613445
